In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import sklearn.feature_extraction.text
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
from sklearn import preprocessing
import re
import string

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

plt.ioff()
pd.set_option('display.max_columns', None)

## Download data from https://www.kaggle.com/crowdflower/twitter-airline-sentiment

In [ ]:
data_frame = pd.read_csv('data/Tweets.csv')

In [ ]:
print("{} Tweets in dataset".format(len(data_frame)))
data_frame.head()

In [ ]:
tweets_text = data_frame['text']
tweets_sentiment = data_frame['airline_sentiment']

In [ ]:
print("Tweet: {}".format(tweets_text[18]))
print("Sentiment: {}".format(tweets_sentiment[18]))

print ""
print("Tweet: {}".format(tweets_text[17]))
print("Sentiment: {}".format(tweets_sentiment[17]))

In [ ]:
print("Tweet: {}".format(tweets_text[3]))
print("Sentiment: {}".format(tweets_sentiment[3]))

In [ ]:
from keras.preprocessing.text import Tokenizer
docs = tweets_text
vocab_size = 5000
t = Tokenizer(nb_words=vocab_size)
t.fit_on_texts(docs)
sequences = t.texts_to_sequences(docs)
print(sequences[0])

In [ ]:
print(sequences[10])
max_review_length = 20
X = sequence.pad_sequences(sequences, maxlen=max_review_length)
print(X[10,...])
y_mapping = {'negative':[1, 0, 0], 'neutral':[0, 1, 0], 'positive':[0, 0, 1]}
y_list = [y_mapping[sentiment.lower()] for sentiment in tweets_sentiment]
y = np.array(y_list)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4., random_state=1)
print(X_train.shape)
print(y_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(X_train[0:])

In [ ]:
embedding_vecor_length = 64
model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, nb_epoch=5, batch_size=64,  validation_data=(X_test, y_test))

In [ ]:
# Final evaluation of the model
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

y_train_pred_class = np.argmax(y_train_pred, axis=1)
y_test_pred_class = np.argmax(y_test_pred, axis=1)
y_train_class = np.argmax(y_train, axis=1)
y_test_class = np.argmax(y_test, axis=1)

train_accuracy = sklearn.metrics.accuracy_score(y_train_class, y_train_pred_class)
test_accuracy = sklearn.metrics.accuracy_score(y_test_class, y_test_pred_class)
print("Train Accuracy {}%".format(train_accuracy*100.0))
print("Test Accuracy {}%".format(test_accuracy*100))

cf_matrix = confusion_matrix(y_test_class, y_test_pred_class)
print("Confusion Matrix")
print(cf_matrix)

In [ ]:
# adapted from:
# https://github.com/scikit-learn/scikit-learn/blob/master/examples/model_selection/plot_confusion_matrix.py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
plot_confusion_matrix(cf_matrix, classes=["Negative", "Neutral", "Positive"],
                      title='Confusion matrix, normalization', normalize=True)
plt.show()